In [331]:
import os
PATH = 'C:/Users/user/DataPreprocessing/ModelingDataset/data'
os.chdir(PATH)

In [332]:
import pandas as pd
import numpy as np
import datetime

### 1. 선박 log에서 가장 가까운 관측소의 날씨 데이터 매칭

In [333]:
shiplog_obs = pd.read_csv('./shiplog_obs.csv',sep=',', encoding = 'cp949',  index_col = 0)
weather = pd.read_csv('./weather.csv', sep=',' , encoding = 'cp949',  index_col = 0)

### 2. shiplog_obs 전처리

In [334]:
shiplog_obs

,insertTime,mmsi,speed,shipLat,shipLon,geometry,nearestObs
0,2022-11-30 16:33:05,371369000,17.1,37.264305,126.245270,POINT (37.2643051147461 126.245269775391),덕적도
1,2022-11-30 16:33:05,440022920,0.0,37.452236,126.610077,POINT (37.4522361755371 126.610076904297),인천
2,2022-11-30 16:33:05,235086166,9.7,37.004253,126.162865,POINT (37.0042533874512 126.162864685059),태안
3,2022-11-30 16:33:05,440022920,0.0,37.452225,126.610085,POINT (37.4522247314453 126.610084533691),인천
4,2022-11-30 16:33:05,441103000,5.3,37.327690,126.510292,POINT (37.3276901245117 126.510292053223),인천송도
...,...,...,...,...,...,...,...
293989,2022-12-30 00:50:02,538009716,11.8,37.445282,126.577316,POINT (37.4452819824219 126.57731628418),인천
293990,2022-12-30 00:50:04,538010048,2.8,37.339527,126.634224,POINT (37.339527130127 126.634223937988),인천송도
293991,2022-12-30 00:50:05,441174000,11.7,37.188816,126.381104,POINT (37.1888160705566 126.381103515625),영흥도
293992,2022-12-30 00:50:07,538010048,2.8,37.339565,126.415787,POINT (37.3395652770996 126.415786743164),영흥도


In [335]:
# insertTime 데이터타입 변환: object -> datetime
# time 포멧 일치 시키기: 초단위 버림 (weather 데이터 1분 주기로 갱신)
shiplog_obs['insertTime'] = pd.to_datetime(shiplog_obs['insertTime']).dt.floor('Min')

In [336]:
#  필요없는 컬럼 제거: geometry
shiplog_obs = shiplog_obs.drop(["geometry"], axis=1)
shiplog_obs

,insertTime,mmsi,speed,shipLat,shipLon,nearestObs
0,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,덕적도
1,2022-11-30 16:33:00,440022920,0.0,37.452236,126.610077,인천
2,2022-11-30 16:33:00,235086166,9.7,37.004253,126.162865,태안
3,2022-11-30 16:33:00,440022920,0.0,37.452225,126.610085,인천
4,2022-11-30 16:33:00,441103000,5.3,37.327690,126.510292,인천송도
...,...,...,...,...,...,...
293989,2022-12-30 00:50:00,538009716,11.8,37.445282,126.577316,인천
293990,2022-12-30 00:50:00,538010048,2.8,37.339527,126.634224,인천송도
293991,2022-12-30 00:50:00,441174000,11.7,37.188816,126.381104,영흥도
293992,2022-12-30 00:50:00,538010048,2.8,37.339565,126.415787,영흥도


### 3. 기상데이터 전처리

In [337]:
# 관측시간 데이터타입 변환: object -> datetime
weather['관측시간'] = pd.to_datetime(weather['관측시간'])
weather

,관측소명,관측시간,위도,경도,조위(cm),풍속(m/s),풍향,기온(℃),기압(hPa)
0,인천송도,2022-11-30 00:01:00,37.338056,126.586111,497.0,10.6,북북서,0.2,1026.9
1,인천송도,2022-11-30 00:02:00,37.338056,126.586111,496.0,9.4,북북서,0.3,1026.7
2,인천송도,2022-11-30 00:03:00,37.338056,126.586111,495.0,9.8,북북서,0.1,1027.0
3,인천송도,2022-11-30 00:04:00,37.338056,126.586111,490.0,8.7,북북서,-0.1,1027.0
4,인천송도,2022-11-30 00:05:00,37.338056,126.586111,490.0,9.3,북북서,-0.1,1027.0
...,...,...,...,...,...,...,...,...,...
306576,덕적도,2022-12-30 23:55:00,37.226333,126.156556,563.0,0.9,북북서,-2.3,1031.7
306577,덕적도,2022-12-30 23:56:00,37.226333,126.156556,561.0,1.0,북서,-2.2,1031.8
306578,덕적도,2022-12-30 23:57:00,37.226333,126.156556,559.0,1.3,서북서,-2.2,1031.8
306579,덕적도,2022-12-30 23:58:00,37.226333,126.156556,558.0,1.4,북서,-2.3,1031.8


In [338]:
# 컬럼명 변경
weather.columns = ['obsName', 'obsTime', 'obsLat', 'obsLon', 'tideLevel', 'windSpeed', 'windDirec', 'temp', 'pressure']

### 4. shiplog_obs, weather join 시키기
- shiplog에 필요한 날씨 데이터만 추출

In [339]:
dt1 = pd.merge(shiplog_obs, weather, left_on = ('nearestObs','insertTime'), right_on = ('obsName','obsTime'), how = 'inner', indicator = True)

In [340]:
dt1.info()
dt1.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292877 entries, 0 to 292876
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   insertTime  292877 non-null  datetime64[ns]
 1   mmsi        292877 non-null  int64         
 2   speed       292877 non-null  float64       
 3   shipLat     292877 non-null  float64       
 4   shipLon     292877 non-null  float64       
 5   nearestObs  292877 non-null  object        
 6   obsName     292877 non-null  object        
 7   obsTime     292877 non-null  datetime64[ns]
 8   obsLat      292877 non-null  float64       
 9   obsLon      292877 non-null  float64       
 10  tideLevel   292877 non-null  float64       
 11  windSpeed   292877 non-null  float64       
 12  windDirec   292877 non-null  object        
 13  temp        292877 non-null  float64       
 14  pressure    292877 non-null  float64       
 15  _merge      292877 non-null  category      
dtypes:

insertTime    0
mmsi          0
speed         0
shipLat       0
shipLon       0
nearestObs    0
obsName       0
obsTime       0
obsLat        0
obsLon        0
tideLevel     0
windSpeed     0
windDirec     0
temp          0
pressure      0
_merge        0
dtype: int64

### 5. join된 dt1 데이터프레임 전처리

In [341]:
# dt1에서 필요없는 컬럼 drop
dt1 = dt1.drop(['nearestObs','obsTime','obsLat','obsLon','_merge'], axis = 1)
dt1 = dt1.drop_duplicates(['insertTime', 'mmsi'], keep = 'first').reset_index(drop=True)

### 6. dt1(shiplog+weather) 데이터프레임과 schedules 데이터프레임 합치기
- 선박용도, 총톤수, 입항일시(실제도착일시) 컬럼을 학습 데이터셋 피쳐에 사용하기 위해서

In [342]:
# 데이터 merge 하려면 키값이 object타입 이어야 한다. 
# dt1와 schedules 합치려면 두 프레임 모두 mmsi 컬럼을 바꿔줘야 함
dt1['mmsi'] = dt1['mmsi'].astype(str)

In [343]:
dt_schedules = pd.read_csv('./schedules.csv', encoding = 'cp949', usecols = ['mmsi', '총톤수', '선박용도','입항일시'])

In [344]:
dt_schedules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mmsi    883 non-null    float64
 1   총톤수     883 non-null    object 
 2   입항일시    883 non-null    object 
 3   선박용도    883 non-null    object 
dtypes: float64(1), object(3)
memory usage: 27.7+ KB


In [345]:
# mmsi 데이터타입 변경: float -> int -> str
dt_schedules['mmsi']= dt_schedules['mmsi'].astype(int)
dt_schedules['mmsi'] = dt_schedules['mmsi'].astype(str)

# totalWeight 데이터타입 변경: object -> int
dt_schedules['총톤수']=dt_schedules['총톤수'].replace(",","", regex= True).astype(int)

In [346]:
dt_schedules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mmsi    883 non-null    object
 1   총톤수     883 non-null    int32 
 2   입항일시    883 non-null    object
 3   선박용도    883 non-null    object
dtypes: int32(1), object(3)
memory usage: 24.3+ KB


In [347]:
# dt_schedules 컬럼 위치 변경
dt_schedules = dt_schedules[['mmsi', '총톤수', '선박용도','입항일시']]
# dt_schedules 컬럼명 변경
dt_schedules.columns = ['mmsi', 'totalWeight', 'shipUse', 'arrivalTime']
dt_schedules

,mmsi,totalWeight,shipUse,arrivalTime
0,371369000,18085,풀컨테이너선,2022-11-30 00:36
1,371369000,18085,풀컨테이너선,2022-12-15 07:05
2,371369000,18085,풀컨테이너선,2022-12-30 02:55
3,441791000,29543,석유제품 운반선,2022-11-30 08:55
4,441791000,29543,석유제품 운반선,2022-12-04 13:45
...,...,...,...,...
878,352001275,2967,일반화물선,2022-12-30 09:00
879,477842900,18820,풀컨테이너선,2022-12-30 13:00
880,613003641,1941,일반화물선,2022-12-30 18:10
881,305977000,17769,풀컨테이너선,2022-12-30 18:55


### 7. dt1(shiplog + weather), dt_schedules(선박정보) 합치기

In [348]:
dataset = pd.merge(dt1, dt_schedules, on = ('mmsi'), how = 'inner', indicator = True)

In [349]:
dataset['arrivalTime'] = pd.to_datetime(dataset['arrivalTime'])
dataset['arrivalTime']

0        2022-11-30 00:36:00
1        2022-12-15 07:05:00
2        2022-12-30 02:55:00
3        2022-11-30 00:36:00
4        2022-12-15 07:05:00
                 ...        
272987   2022-12-30 02:45:00
272988   2022-12-09 08:55:00
272989   2022-12-30 02:45:00
272990   2022-12-09 08:55:00
272991   2022-12-30 02:45:00
Name: arrivalTime, Length: 272992, dtype: datetime64[ns]

In [350]:
dataset

,insertTime,mmsi,speed,shipLat,shipLon,obsName,tideLevel,windSpeed,windDirec,temp,pressure,totalWeight,shipUse,arrivalTime,_merge
0,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,덕적도,176.0,2.6,북서,-3.1,1032.6,18085,풀컨테이너선,2022-11-30 00:36:00,both
1,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,덕적도,176.0,2.6,북서,-3.1,1032.6,18085,풀컨테이너선,2022-12-15 07:05:00,both
2,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,덕적도,176.0,2.6,북서,-3.1,1032.6,18085,풀컨테이너선,2022-12-30 02:55:00,both
3,2022-12-29 23:23:00,371369000,10.0,36.960186,126.101601,태안,517.0,2.2,서,2.7,1031.7,18085,풀컨테이너선,2022-11-30 00:36:00,both
4,2022-12-29 23:23:00,371369000,10.0,36.960186,126.101601,태안,517.0,2.2,서,2.7,1031.7,18085,풀컨테이너선,2022-12-15 07:05:00,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272987,2022-12-30 00:49:00,441174000,11.8,37.186699,126.378830,영흥도,426.0,1.9,북,0.6,1030.9,16717,풀컨테이너선,2022-12-30 02:45:00,both
272988,2022-12-30 00:48:00,441174000,11.7,37.185482,126.377480,영흥도,428.0,1.2,북,0.7,1030.9,16717,풀컨테이너선,2022-12-09 08:55:00,both
272989,2022-12-30 00:48:00,441174000,11.7,37.185482,126.377480,영흥도,428.0,1.2,북,0.7,1030.9,16717,풀컨테이너선,2022-12-30 02:45:00,both
272990,2022-12-30 00:50:00,441174000,11.7,37.188816,126.381104,영흥도,424.0,1.1,북,0.7,1030.9,16717,풀컨테이너선,2022-12-09 08:55:00,both


In [351]:
# dataset에서 필요없는 컬럼 제거
dataset.drop(['obsName','_merge'], axis = 1, inplace = True)

In [352]:
# dataset에서 중복되는 데이터 삭제
dataset.drop_duplicates(['mmsi', 'insertTime', 'arrivalTime'], keep = 'first')

,insertTime,mmsi,speed,shipLat,shipLon,tideLevel,windSpeed,windDirec,temp,pressure,totalWeight,shipUse,arrivalTime
0,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,176.0,2.6,북서,-3.1,1032.6,18085,풀컨테이너선,2022-11-30 00:36:00
1,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,176.0,2.6,북서,-3.1,1032.6,18085,풀컨테이너선,2022-12-15 07:05:00
2,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,176.0,2.6,북서,-3.1,1032.6,18085,풀컨테이너선,2022-12-30 02:55:00
3,2022-12-29 23:23:00,371369000,10.0,36.960186,126.101601,517.0,2.2,서,2.7,1031.7,18085,풀컨테이너선,2022-11-30 00:36:00
4,2022-12-29 23:23:00,371369000,10.0,36.960186,126.101601,517.0,2.2,서,2.7,1031.7,18085,풀컨테이너선,2022-12-15 07:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
272987,2022-12-30 00:49:00,441174000,11.8,37.186699,126.378830,426.0,1.9,북,0.6,1030.9,16717,풀컨테이너선,2022-12-30 02:45:00
272988,2022-12-30 00:48:00,441174000,11.7,37.185482,126.377480,428.0,1.2,북,0.7,1030.9,16717,풀컨테이너선,2022-12-09 08:55:00
272989,2022-12-30 00:48:00,441174000,11.7,37.185482,126.377480,428.0,1.2,북,0.7,1030.9,16717,풀컨테이너선,2022-12-30 02:45:00
272990,2022-12-30 00:50:00,441174000,11.7,37.188816,126.381104,424.0,1.1,북,0.7,1030.9,16717,풀컨테이너선,2022-12-09 08:55:00


In [353]:
# insertTIme이 arrivalTime 보다 늦은 경우 삭제
dataset = dataset.query("insertTime < arrivalTime")

# insertTime, mmsi가 중복되는 행은 첫번째를 제외하고 삭제
dataset = dataset.drop_duplicates(['insertTime', 'mmsi'], keep = 'first').reset_index(drop=True)
dataset

,insertTime,mmsi,speed,shipLat,shipLon,tideLevel,windSpeed,windDirec,temp,pressure,totalWeight,shipUse,arrivalTime
0,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,176.0,2.6,북서,-3.1,1032.6,18085,풀컨테이너선,2022-12-15 07:05:00
1,2022-12-29 23:23:00,371369000,10.0,36.960186,126.101601,517.0,2.2,서,2.7,1031.7,18085,풀컨테이너선,2022-12-30 02:55:00
2,2022-12-29 23:39:00,371369000,9.8,36.990513,126.137215,487.0,1.4,남서,2.4,1031.6,18085,풀컨테이너선,2022-12-30 02:55:00
3,2022-12-29 23:37:00,371369000,9.8,36.988640,126.134674,490.0,2.2,서남서,2.4,1031.6,18085,풀컨테이너선,2022-12-30 02:55:00
4,2022-12-29 23:40:00,371369000,9.7,36.993187,126.140976,485.0,1.9,남서,2.4,1031.6,18085,풀컨테이너선,2022-12-30 02:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51446,2022-12-30 00:46:00,441174000,11.3,37.179001,126.370865,433.0,1.7,북,0.9,1030.9,16717,풀컨테이너선,2022-12-30 02:45:00
51447,2022-12-30 00:47:00,441174000,11.5,37.181301,126.373184,430.0,1.9,북,0.8,1030.9,16717,풀컨테이너선,2022-12-30 02:45:00
51448,2022-12-30 00:49:00,441174000,11.8,37.186699,126.378830,426.0,1.9,북,0.6,1030.9,16717,풀컨테이너선,2022-12-30 02:45:00
51449,2022-12-30 00:48:00,441174000,11.7,37.185482,126.377480,428.0,1.2,북,0.7,1030.9,16717,풀컨테이너선,2022-12-30 02:45:00


### 8. 선박 현재위치에서 도착지까지 거리 계산
- 학습에 영향이 많을 거라 생각
- rest_dis 컬럼 생성
- from haversine import haversine, Unit

In [354]:
from haversine import haversine, Unit

def rest_dist():
    # 도착지: 인천항 고정 
    arrival_port = (37.440515,126.601098)
    
    shipPosition = []
    for i in range(len(dataset)):
        position = (dataset['shipLat'][i],dataset['shipLon'][i])
        shipPosition.append(position)
        
    restDist = []
    for i in range(len(shipPosition)):
        try:
            rest_dist = haversine(arrival_port, shipPosition[i], unit = 'm')
            restDist.append(rest_dist)
        except:
            "None" # 이상치 null 처리 
    dataset['rest_dist'] = pd.DataFrame(restDist)
    dataset.dropna(subset = ['rest_dist'], axis = 0, inplace = True)
    return dataset

In [355]:
dataset = rest_dist()

In [356]:
dataset

,insertTime,mmsi,speed,shipLat,shipLon,tideLevel,windSpeed,windDirec,temp,pressure,totalWeight,shipUse,arrivalTime,rest_dist
0,2022-11-30 16:33:00,371369000,17.1,37.264305,126.245270,176.0,2.6,북서,-3.1,1032.6,18085,풀컨테이너선,2022-12-15 07:05:00,37055.892919
1,2022-12-29 23:23:00,371369000,10.0,36.960186,126.101601,517.0,2.2,서,2.7,1031.7,18085,풀컨테이너선,2022-12-30 02:55:00,69352.916750
2,2022-12-29 23:39:00,371369000,9.8,36.990513,126.137215,487.0,1.4,남서,2.4,1031.6,18085,풀컨테이너선,2022-12-30 02:55:00,64739.187778
3,2022-12-29 23:37:00,371369000,9.8,36.988640,126.134674,490.0,2.2,서남서,2.4,1031.6,18085,풀컨테이너선,2022-12-30 02:55:00,65043.247084
4,2022-12-29 23:40:00,371369000,9.7,36.993187,126.140976,485.0,1.9,남서,2.4,1031.6,18085,풀컨테이너선,2022-12-30 02:55:00,64297.611382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51388,2022-12-29 23:41:00,441174000,10.1,37.028133,126.192680,483.0,2.6,남남서,2.3,1031.6,16717,풀컨테이너선,2022-12-30 02:45:00,35499.387933
51389,2022-12-29 23:42:00,441174000,10.8,37.031849,126.197449,482.0,2.0,남서,2.3,1031.6,16717,풀컨테이너선,2022-12-30 02:45:00,35172.042374
51390,2022-12-29 23:43:00,441174000,11.1,37.033585,126.199715,480.0,1.8,서남서,2.2,1031.6,16717,풀컨테이너선,2022-12-30 02:45:00,34393.687393
51391,2022-12-29 23:48:00,441174000,11.5,37.043816,126.213150,470.0,2.1,남서,2.3,1031.6,16717,풀컨테이너선,2022-12-30 02:45:00,34573.073957


In [357]:
arrival = pd.to_datetime(dataset['arrivalTime'])
timestamp = []
for time in arrival:
    timestamp.append(int(time.timestamp()))
dataset['arrivalTime'] = timestamp

In [360]:
# insert = pd.to_datetime(dataset['insertTime'])
# timestamp = []
# for time in insert:
#     timestamp.append(int(time.timestamp()))
# dataset['insertTime'] = timestamp

In [361]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51393 entries, 0 to 51392
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   insertTime   51393 non-null  int64  
 1   mmsi         51393 non-null  object 
 2   speed        51393 non-null  float64
 3   shipLat      51393 non-null  float64
 4   shipLon      51393 non-null  float64
 5   tideLevel    51393 non-null  float64
 6   windSpeed    51393 non-null  float64
 7   windDirec    51393 non-null  object 
 8   temp         51393 non-null  float64
 9   pressure     51393 non-null  float64
 10  totalWeight  51393 non-null  int32  
 11  shipUse      51393 non-null  object 
 12  arrivalTime  51393 non-null  int64  
 13  rest_dist    51393 non-null  float64
dtypes: float64(8), int32(1), int64(2), object(3)
memory usage: 5.7+ MB


In [362]:
dataset['shipUse'].unique()

array(['풀컨테이너선', '자동차운반선', 'LPG 운반선', '일반화물선', '기타 예선', '석유제품 운반선',
       '시멘트운반선', '케미칼 운반선', '견인용예선', '급유선', '산물선(벌크선)', '원유운반선',
       'LNG 운반선', '세미(혼재)컨테이너선', '화객선', '압항 예선'], dtype=object)

### 10. time 월 / 일 / 요일 / 시 / 분 분리
- insertTime => LSTM 모델 인덱스로 사용하기 위해서 분리하지 않기로 결정
- arrivalTime

In [58]:
# import datetime
# m = []
# d = []
# w = []
# h = []
# mm = []
# def time_split(time):
#     for i in range(len(time)):
#         m.append(time[i].month)
#         d.append(time[i].day)
#         h.append(time[i].hour)
#         mm.append(time[i].minute)
#         w.append(time[i].weekday())
#     return 

In [59]:
# dataset['arrivalTime'] = pd.to_datetime(dataset['arrivalTime'])

# # arrivalTime 분리
# time_split(dataset['arrivalTime'])
# dataset['arrivalMonth'] = m
# dataset['arrivalDate'] = d
# dataset['arrivalHour'] = h
# dataset['arrivalMinute'] = mm
# dataset['arrivalDay'] = w

In [60]:
# dataset.drop(['insertTime'], axis = 1, inplace = True)

In [363]:
dataset.to_csv("./dataset_pre1.csv", sep = ',',encoding = 'cp949')